# Adjust the settings in the next cell if necessary

In [1]:
import pytesseract

# COMMENT IF NOT USING WINDOWS (i.e., add # before both lines)
pytesseract.pytesseract.tesseract_cmd = "C:/Program Files/Tesseract-OCR/tesseract"
TESSDATA_PREFIX = r"C:\Program Files\Tesseract-OCR"

# Change to False if you always want your actual main stat value
assume_max_lv_gear = True

# Screenshot path
sh_path = 'screenshots/'

# Do not change anything below here

In [2]:
from PIL import Image
from pytesseract import image_to_string

def process(k,img):
    if  k == 'plus' or k == 'level':
        # Since level and plus are read off the gear icon, there is a lot of noise. Thus, we need an iterative process to try to extract a value
        thresh = cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU
        for low in [81,100,125]:
            proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_RGB2GRAY), low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
            data = image_to_string(Image.fromarray(proc), lang='eng', config='--psm 7').replace('+b','6').replace('>','0')
            if any(i.isdigit() for i in data): return data
            elif low == 125: data = str(0)
    else:
        thresh = cv2.THRESH_BINARY_INV
        low = 70
        proc = cv2.cvtColor(cv2.medianBlur(cv2.threshold(cv2.cvtColor(cv2.resize(img, (0, 0), fx=5, fy=5), cv2.COLOR_BGR2GRAY), low, 255, thresh)[1], 3), cv2.COLOR_GRAY2RGB)
        data = image_to_string(Image.fromarray(proc), lang='eng', config='--psm 6').replace('Rina','Ring').replace('Edic','Epic').replace('Enic','Epic')
    return data

def stat_converter(stat):
    result = ''
    if 'attack' in stat.lower():
        result = 'Atk'
        if '%' in stat: result += 'P'
    if 'health' in stat.lower():
        result = 'HP'
        if '%' in stat: result += 'P'
    if 'defense' in stat.lower():
        result = 'Def'
        if '%' in stat: result += 'P'
    if 'speed' in stat.lower(): result = 'Spd'
    if 'chance' in stat.lower(): result = 'CChance'
    if 'damage' in stat.lower(): result = 'CDmg'
    if 'effectiveness' in stat.lower(): result = 'Eff'
    if 'resistance' in stat.lower(): result = 'Res'
    return result
  

def digit_filter(val):
    try: return int(''.join(filter(str.isdigit,val)))
    except ValueError: return 0

def char_filter(val):
    try: return ''.join(filter(str.isalpha,val)).capitalize()
    except ValueError: return ''

def max_stat(data,item):
    stat = stat_converter(data)
    val = digit_filter(data) # Begin by setting val = actual value
    if item['ability'] < 15: # Only change stats on items where they need to be increased
        if item['level'] in range(58,73):
            if stat == 'CChance': val = 45
            elif stat == 'CDmg': val = 55
            elif stat == 'Spd': val = 35
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 50
            elif stat == 'HP': val = 2295 # Not exactly right, finer-grained scaling
            elif stat == 'Def': val = 250 # Not exactly right, finer-grained scaling
            elif stat == 'Atk': val = 425 # Not exactly right, finer-grained scaling
        elif item['level'] in range(74,86):
            if stat == 'CChance': val = 55
            elif stat == 'CDmg': val = 65
            elif stat == 'Spd': val = 40
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 60
            elif stat == 'HP': val = 2700 # Not exactly right, finer-grained scaling
            elif stat == 'Def': val = 300 # Not exactly right, finer-grained scaling
            elif stat == 'Atk': val = 500 # Not exactly right, finer-grained scaling
        elif item['level'] in range(87,89):
            if stat == 'CChance': val = 60
            elif stat == 'CDmg': val = 70
            elif stat == 'Spd': val = 45
            elif item['slot'] == ('Necklace' or 'Ring' or 'Boots'): val = 65
            elif stat == 'HP': val = 2765
            elif stat == 'Def': val = 310
            elif stat == 'Atk': val = 515
    return val

In [3]:
from glob import glob
from string import ascii_lowercase, digits
import random
import json
import cv2

# Format the json for the optimizer
export = {'processVersion':'1','heroes':[],'items':[]}
temp_list = []

print('Beginning export process...')

# Iterate through the screenshots
filenames = glob(sh_path+'*.png')
filenames.extend(glob(sh_path+'*.jpg'))
filenames.extend(glob(sh_path+'*.jpeg'))

did_break = False
errors = 0
for n,name in enumerate(filenames):
    
    # The height of the item box changes depending on the length of the item and set descriptions,
    # we have to crop the top and bottom info separately in order to ensure the OCR boxes within these areas
    # remain in fixed locations. we then process the top and bottom info independently.
    
    print('currently processing '+name.replace('screenshots\\',''))
    img = cv2.imread(name)
    if img.shape[0] != 1080 or img.shape[1] != 2200:
        print('ERROR: This screenshots is not 2200x1080!')
        print('Please retake the screenshot in Nox or LDPlayer with the internal resolution set to 2200x1080')
        did_break = True
        break
    
    # Top box
    temp_top = cv2.imread('e7/top.jpg',0)
    a, b, c, max_loc = cv2.minMaxLoc(cv2.matchTemplate(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), temp_top, cv2.TM_CCOEFF_NORMED))
    top_box = img[max_loc[1]:max_loc[1]+160,740:1190] # Fixed width, then crop 160 pixels from top triangle

    # Bottom box
    temp_bot = cv2.imread('e7/bottom.jpg',0)
    a, b, c, max_loc = cv2.minMaxLoc(cv2.matchTemplate(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), temp_bot, cv2.TM_CCOEFF_NORMED))
    bottom_box = img[max_loc[1]+25:max_loc[1]+360,740:1190] # Fixed width, shift down 25 from divider, then crop 335 pixels deep

    # Setup item dictionary
    item = {'locked':False,'efficiency':0}

    # Process top image
    top_coords = {'type': [[20,70],[172,432]],
                  'level': [[19,44],[37,66]],
                  'plus': [[11,34],[139,168]]}
    
    for k in top_coords.keys():
        data = process(k,top_box[top_coords[k][0][0]:top_coords[k][0][1],top_coords[k][1][0]:top_coords[k][1][1]])
        if k == 'type':
            item['rarity'] = char_filter(data.split(' ')[0])
            item['slot'] = char_filter(data.split(' ')[1].split('\n')[0])
            if len(item['rarity']) == 0: print('Error: no rarity detected')
            if len(item['slot']) == 0: print('Error: no slot detected')
        if k == 'level':
            item['level'] = digit_filter(data.replace('S','5').replace('B','8').replace('a','8'))
            if item['level'] == 0: item['level'] = 1
        if k == 'plus':
            item['ability'] = digit_filter(data.replace('S','5').replace('B','8').replace('a','8'))

    #print("ilvl", item['level'],"/ enhanced to +"+str(item['ability'])+" / slot:",item['slot']," / rarity:",item['rarity'])
    
    # Process bottom image
    bot_coords = {'main': [[8,70],[65,435]],
                  'subs': [[98,255],[25,435]],
                  'set': [[280,340],[76,435]]}
    
    for k in bot_coords.keys():
        data = process(k,bottom_box[bot_coords[k][0][0]:bot_coords[k][0][1],bot_coords[k][1][0]:bot_coords[k][1][1]])
        if k == 'main':
            stat = stat_converter(data)
            if assume_max_lv_gear is True: item['mainStat'] = [stat,max_stat(data,item)]
            else: item['mainStat'] = [stat,digit_filter(data)]     
        
        if k == 'subs':
            for n,entry in enumerate(data.split('\n')):
                if n < 4:
                    stat = stat_converter(entry)
                    val = digit_filter(entry.replace('T','7'))
                    if len(stat) > 0 and val != 0: item['subStat'+str(n+1)] = [stat,val]
        if k == 'set':
            item['set'] = char_filter(data.split(' Set')[0])
            if len(item['set']) == 0: print('Error: no set detected')
    
    # Check to see if there are duplicates and if it has a valid slot and rarity entry, and if not, give item unique ID and place in 'export'
    if item not in temp_list and len(item['slot']) > 0 and len(item['rarity']) > 0:
        temp_list.append(item)
        item['id'] = 'jt'+''.join(random.choice(digits+ascii_lowercase) for _ in range(6))
        export['items'].append(item)
        print(len(export['items']),"exported out of",len(filenames)-errors,'valid items \n')
    else:
        errors += 1
        print('Item not exported because of fatal errors in processing it \n')
    
    
    
    
# Export dictionary to json for importing into optimizer if everything completed properly
if not did_break:
    with open('exported_gear.json', 'w') as f: json.dump(export, f)
    print('JSON file finished!')

Beginning export process...
currently processing Screenshot_20190322-115231.jpg
1 exported out of 1 valid items 

JSON file finished!
